In [1]:
import pandas as pd
import seaborn as sns
wd = "/Users/timvigers/Dropbox/Work/Petter Bjornstad/Pima/Master data/Raw data/"
# wd = "path/to/Laura's/folder"

# Vital Status

In [2]:
# Import
vital_status = pd.read_csv(wd+'NelsonVitalStatusDEC_DATA_2021-02-04_1616.csv')
# Delete empty rows
vital_status = vital_status[vital_status['end_stage_renal_disease_form_complete']!=0]
# Collapse rows - get first non-NA element in each column by record ID
vital_status = vital_status.groupby('record_id',as_index=False).first()
# Get columns of interest - ask Rob and Helen for a list of these
vital_status = vital_status[['esrd_start_date','dod']]

## Data Description

In [3]:
vital_status.describe()

,esrd_start_date,dod
count,93,185
unique,92,184
top,1999-06-04,1999-09-18
freq,2,2


# Group 4

In [49]:
# Import
group4 = pd.read_csv(wd+'Group4UofMRemodel112_DATA_2021-02-04_1617.csv',low_memory=False)
# Index by record ID
group4 = group4.set_index(['record_id','redcap_event_name'])
# Get GFR
date_cols = [col for col in group4.columns if 'visitdate' in col]
gfr_cols = [col for col in group4.columns if 'gfr' in col and "_mv" not in col]
gfr = group4[date_cols+gfr_cols]
gfr.dropna(axis=0,subset=gfr_cols,how='all').dropna(axis=1,how='all')

f4visitdate f4_visitdate  f4_visitdate_mv  \
record_id redcap_event_name                                                
752       rc_interval_0_arm_1   1994-12-01   1994-12-01              NaN   
          rc_interval_6_arm_1   1995-08-25   1995-08-25              NaN   
          rc_interval_12_arm_1  1995-12-12   1995-12-12              NaN   
          rc_interval_18_arm_1  1996-06-25   1996-06-25              NaN   
          rc_interval_42_arm_1  1998-04-08   1998-04-08              NaN   
...                                    ...          ...              ...   
4996      rc_interval_54_arm_1  1999-09-01   1999-09-01              NaN   
          rc_interval_60_arm_1  1999-12-10   1999-12-10              NaN   
          rc_interval_66_arm_1  2000-06-06   2000-06-06              NaN   
          rc_interval_72_arm_1  2000-12-07   2000-12-07              NaN   
8023      rc_interval_0_arm_1   1995-01-10   1995-01-10              NaN   

                               f4meds_visitdate f5visitdate f6visitdate  \
record_id redcap_event_name                                               
752       rc_interval_0_arm_1        1994-12-01  1994-12-01  1994-12-01   
          rc_interval_6_arm_1        1995-08-25  1995-08-25  1995-08-25   
          rc_interval_12_arm_1       1995-12-12  1995-12-12  1995-12-12   
          rc_interval_18_arm_1       1996-06-25  1996-06-25  1996-06-25   
          rc_interval_42_arm_1       1998-04-08  1998-04-08  1998-04-08   
...                                         ...         ...         ...   
4996      rc_interval_54_arm_1       1999-09-01  1999-09-01  1999-09-01   
          rc_interval_60_arm_1       1999-12-10  1999-12-10  1999-12-10   
          rc_interval_66_arm_1       2000-06-06  2000-06-06  2000-06-06   
          rc_interval_72_arm_1       2000-12-07  2000-12-07  2000-12-07   
8023      rc_interval_0_arm_1        1995-01-10  1995-01-10  1995-01-10   

                               f8visitdate  f8visitdate_mv f9visitdate  \
record_id redcap_event_name                                              
752       rc_interval_0_arm_1   1994-12-01             NaN  1994-12-01   
          rc_interval_6_arm_1   1995-08-25             NaN  1995-08-25   
          rc_interval_12_arm_1  1995-12-12             NaN  1995-12-12   
          rc_interval_18_arm_1  1996-06-25             NaN  1996-06-25   
          rc_interval_42_arm_1  1998-04-08             NaN  1998-04-08   
...                                    ...             ...         ...   
4996      rc_interval_54_arm_1  1999-09-01             NaN  1999-09-01   
          rc_interval_60_arm_1  1999-12-10             NaN  1999-12-10   
          rc_interval_66_arm_1  2000-06-06             NaN  2000-06-06   
          rc_interval_72_arm_1  2000-12-07             NaN  2000-12-07   
8023      rc_interval_0_arm_1   1995-01-10             NaN  1995-01-10   

                                f9visitdate_mv  ... f17visitdate f18visitdate  \
record_id redcap_event_name                     ...                             
752       rc_interval_0_arm_1              NaN  ...          NaN   1994-12-01   
          rc_interval_6_arm_1              NaN  ...          NaN          NaN   
          rc_interval_12_arm_1             NaN  ...          NaN          NaN   
          rc_interval_18_arm_1             NaN  ...          NaN   1996-06-25   
          rc_interval_42_arm_1             NaN  ...          NaN          NaN   
...                                        ...  ...          ...          ...   
4996      rc_interval_54_arm_1             NaN  ...          NaN          NaN   
          rc_interval_60_arm_1             NaN  ...          NaN          NaN   
          rc_interval_66_arm_1             NaN  ...          NaN          NaN   
          rc_interval_72_arm_1             NaN  ...          NaN          NaN   
8023      rc_interval_0_arm_1              NaN  ...          NaN          NaN   

                               plvisitdate slvisi

# Pickle everything!